# Submission Guidelines

## 1. Complete the Jupyter Notebook
- Fill in all code sections marked with `TODO` in the skeleton (class `Decision_tree`).  
- Implement core functions like `generate_tree`, `combined_metric`, and `metric`.  
- The notebook should:
  - Train and evaluate the model with both entropy and Gini as splitting metrics.
  - Report accuracy for each `min_entropy` value.
  - Evaluate different model choices and report results on the test set.

## 2. Include All Outputs
- Run all cells before submission.  
- Validation and test accuracy results should be clearly printed for all settings.  
- Optional diagnostics (e.g., confusion matrix, tree depth) must render if included.  
- **Missing outputs will result in a penalty.**

## 3. Submit the Following:
- A zipped folder with:
  - Your `.ipynb` notebook (final results)
  - Your edited class `Decision_tree` implementation

**Do NOT** submit the dataset files, sample notebooks, or saved image files (`.png`).  

## 4. Code Clarity & Documentation
- Use clear and descriptive variable names.  
- Add comments where necessary, especially if you deviate from the skeleton.  
- Follow good Python style (PEP-8) for spacing and readability.  

⚠️ **Failure to adhere to these guidelines may result in deductions.**

---

## Decision Tree for Digit Classification

This assignment focuses on implementing a binary-feature decision tree for classifying handwritten digits using the `optdigits` dataset. You will recursively build the tree, compute entropy or Gini metrics, and evaluate performance using validation accuracy.

### Rubric (Total = 40 points):

---

### Tree Implementation – 26 points total

**Tree Construction**  
- Recursive splitting via `generate_tree(...)` – **5 points**  
- Correct use of stopping condition (`min_entropy`) – **4 points**  

**Metric Computation**  
- `metric(...)` correctly computes entropy – **5 points**  
- `combined_metric(...)` uses weighted average – **4 points**

**Predict Function**  
- `predict(...)` correctly traverses the tree and predicts labels – **4 points**  
- Correct feature selection via `select_feature(...)` – **4 points**

---

### Problem A: Hyperparameter Sweep – 6 points total

- Finding and explaining the best `min_entropy ∈ {0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1, 2.0}` – **3 points**  
- Explaining model complexity – **3 points**  

---

### Problem B: Analysis & Report – 8 points total

- Implementing Gini impurity metric – **5 points**  
- Compare entropy vs Gini results – **3 points**  

---

### Notes:
- All code to be implemented is marked with `TODO` in the class `Decision_tree`.
- Use only binary features in all splits.
- You may use external libraries (like `numpy`) but not sklearn's tree implementation.
- Be sure to normalize your input only if explicitly instructed — in this task, it is not required.


In [29]:
import numpy as np
np.random.seed(100)

In [30]:
class Tree_node:
    """Data structure for nodes in the decision-tree"""
    
    def __init__(self):
        self.feature = None      # Index of the selected feature (for non-leaf node)
        self.label = None        # Class label (for leaf node), if not leaf node, label will be None
        self.left_child = None   # Left child node
        self.right_child = None  # Right child node
        

In [31]:
class Decision_tree:
    """Decision tree with binary features"""
    
    def __init__(self, min_entropy, metric = "entropy"):
        self.metricname = metric
        self.min_entropy = min_entropy
        self.root = None

    def fit(self, train_x, train_y):
        # Construct the decision-tree with recursion
        self.root = self.generate_tree(train_x,train_y)

    def predict(self, test_x):
        # Iterate through all samples
        prediction = []
        for i in range(len(test_x)):
            curr_data = test_x[i]
            
            # Traverse the decision-tree based on the features of the current sample
            curr_node = self.root
            while True:
                if curr_node.label != None:
                    break
                elif curr_node.feature == None:
                    print("You haven't selected the feature yet")
                    exit()
                else:
                    if curr_data[curr_node.feature] == 0:
                        curr_node = curr_node.left_child
                    else:
                        curr_node = curr_node.right_child
                        
            prediction.append(curr_node.label)
        prediction = np.array(prediction)
        
        return prediction

    #
    # TODO
    #
    # Use recursion to build up the tree. Starting from the root node, you can call itself 
    # to determine what is the left_child and what is the right_child. 
    #
    # Return: 
    #   curr_node: the current tree node you create (Type: Tree_Node)
    #
    def generate_tree(self, data, label):
        # Initialize the current tree node
        curr_node = Tree_node()

        # Compute the node entropy or gini index and determine if the current node is a 
        # leaf node. Specifically, if entropy/gini_index (ie, self.metric(label)) < min_entropy), 
        # determine what will be the label (by choosing the label with the largest count) 
        # for this leaf node  and directly return the leaf node 
        node_entropy = self.metric(label)
        if node_entropy < self.min_entropy:
             # Assign majority class label to the leaf
            unique_labels, counts = np.unique(label, return_counts=True)
            curr_node.label = unique_labels[np.argmax(counts)]
            #
            # TODO
            #
            # Add your lines here

            return curr_node

        #
        # TODO
        #
        # Select the feature that will best split the current non-leaf node assign the 
        # feature index to curr_node.feature
        #selected_feature = 0  # Placeholder
        # Select best feature to split on
        selected_feature = self.select_feature(data, label)
        curr_node.feature = selected_feature

        # Split the data based on the selected feature.
        #
        # If the selected feature of the data equals to 0, assign the data and corresponding 
        # point to left_x, left,y otherwise assinged to right_x, right_y
        select_x = data[:, selected_feature]
        left_x = data[select_x == 0]
        left_y = label[select_x == 0]
        right_x = data[select_x == 1]
        right_y = label[select_x == 1]

        #
        # TODO
        #
        # Determine curr_node.left_child and curr_node.right_child by call itself, with left_x, 
        # left_y and right_x, right_y
        # Recursively create subtrees
        curr_node.left_child = self.generate_tree(left_x, left_y)
        curr_node.right_child = self.generate_tree(right_x, right_y)

        return curr_node

    # Select the feature that maximize the information gains
    #
    # Return: 
    #   best_feat: which is the index of the feature
    def select_feature(self, data, label):
        best_feat = 0
        min_entropy = float("inf")

        # Iterate through all features and compute their corresponding entropy 
        for i in range(len(data[0])):
            # Split data based on i-th feature
            split_x = data[:, i]
            left_y = label[split_x == 0]
            right_y = label[split_x == 1]

            # Compute the combined entropy which weightedly combine the entropy/gini of 
            # left_y and right_y
            curr_entropy = self.combined_metric(left_y, right_y)

            # Select the feature with minimum entropy (set best_feat)
            if curr_entropy < min_entropy:
                min_entropy = curr_entropy
                best_feat = i

        return best_feat

    #
    # TODO
    #
    # Weightedly combine the entropy/gini of left_y and right_y. The weights are 
    #
    #  [
    #    len(left_y) / (len(left_y) + len(right_y)), 
    #    len(right_y) / (len(left_y) + len(right_y))
    #  ] 
    #
    # Return: 
    #   result
    def combined_metric(self,left_y,right_y):
        # Compute the entropy of a potential split
        #result = 0
        
        # 
        # TODO
        #
        # Add your lines here. Hint: Use self.metric(label) to compute the entropy/gini_index
        # Compute the entropy or Gini index of a potential split
        total = len(left_y) + len(right_y)
        weight_left = len(left_y) / total
        weight_right = len(right_y) / total
    
        # Use self.metric() to compute impurity of each side and combine
        result = weight_left * self.metric(left_y) + weight_right * self.metric(right_y)
        return result

    #
    # TODO
    #
    # Compute entropy/gini_index based on the labels. The entropy is
    # 
    #   entropy = -sum_i p_i * log2(p_i + 1e-15) 
    #
    # Where we add 1e-15 inside the log when computing the entropy to prevent numerical 
    # issues. The Gini Index is 
    # 
    #   gini_index = 1 - sum_i p_i^2
    #
    def metric(self, label):
        result = 0
        if self.metricname == "entropy":
            class_label, count = np.unique(label, return_counts = True)
            count = count / len(label)
        
            # TODO
            # Add a line here
            #Calculate entropy
            result = -np.sum(count * np.log2(count + 1e-15))
            
        elif self.metricname == "gini_index":
            class_label, count = np.unique(label, return_counts = True)
            count = count / len(label)
            
            # 
            # TODO
            #calcualte gini index
            # Add a line here
            result = 1 - np.sum(count ** 2)
            
        return result

## Load in Data

In [32]:
# Training data
train_data = np.genfromtxt("optdigits_train.txt", delimiter = ",")
train_x = train_data[:, :-1]
train_y = train_data[:, -1].astype("int")

# Validation data
valid_data = np.genfromtxt("optdigits_valid.txt", delimiter = ",")
valid_x = valid_data[:, :-1]
valid_y = valid_data[:, -1].astype("int")

# Test data
test_data = np.genfromtxt("optdigits_test.txt", delimiter = ",")
test_x = test_data[:, :-1]
test_y = test_data[:, -1].astype("int")

print("train_x:", train_x.shape)
print("train_y:", train_y.shape)
print("valid_x:", valid_x.shape)
print("valid_y:", valid_y.shape)
print("test_x:",  test_x.shape)
print("test_y:",  test_y.shape)

train_x: (3000, 64)
train_y: (3000,)
valid_x: (562, 64)
valid_y: (562,)
test_x: (562, 64)
test_y: (562,)


## Problem A

Implement a Decision Tree with the minimum node entropy $\theta$=0.01, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0 and 4.0. The minimum node entropy is used to determine the leaf nodes, i.e., a node is a leaf node if its node entropy is lower than the selected minimum node entropy.

Report the training and validation accuracy by different $\theta$.

In [33]:
# Problem A

# Experiment with different settings of minimum node entropy
candidate_min_entropy = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1, 2.0]

valid_accuracy = []
for i, min_entropy in enumerate(candidate_min_entropy):
    # Initialize and fit.
    clf = Decision_tree(min_entropy = min_entropy, metric = "entropy")
    clf.fit(train_x, train_y)
    
    predictions_train = clf.predict(train_x)
    predictions_val = clf.predict(valid_x)
    
    curr_train_accuracy = np.count_nonzero(predictions_train.reshape(-1) == train_y.reshape(-1)) / len(train_x)
    curr_valid_accuracy = np.count_nonzero(predictions_val.reshape(-1) == valid_y.reshape(-1)) / len(valid_x)
    valid_accuracy.append(curr_valid_accuracy)
    
    print(
        f"Training/validation accuracy for minimum node entropy {candidate_min_entropy[i]} "
        f"is {curr_train_accuracy:.3f} / {curr_valid_accuracy}"
    )

# Select the best minimum node entropy and use it to train the model
best_entropy = candidate_min_entropy[np.argmax(valid_accuracy)]
clf = Decision_tree(min_entropy = best_entropy)
clf.fit(train_x, train_y)

# Evaluate on test data
predictions = clf.predict(test_x)
accuracy = np.count_nonzero(predictions.reshape(-1) == test_y.reshape(-1)) / len(test_x)

print(f"Test accuracy with minimum node entropy {best_entropy} is {accuracy:.3f}")

Training/validation accuracy for minimum node entropy 0.01 is 1.000 / 0.8629893238434164
Training/validation accuracy for minimum node entropy 0.05 is 0.999 / 0.8629893238434164
Training/validation accuracy for minimum node entropy 0.1 is 0.997 / 0.8647686832740213
Training/validation accuracy for minimum node entropy 0.2 is 0.990 / 0.8665480427046264
Training/validation accuracy for minimum node entropy 0.5 is 0.963 / 0.8629893238434164
Training/validation accuracy for minimum node entropy 0.8 is 0.919 / 0.8558718861209964
Training/validation accuracy for minimum node entropy 1 is 0.871 / 0.8398576512455516
Training/validation accuracy for minimum node entropy 2.0 is 0.596 / 0.599644128113879
Test accuracy with minimum node entropy 0.2 is 0.872


#### Question
Which $\theta$ performs best? Report the accuracy on the test set using this $\theta$?

[ANSWER GOES HERE]
The best performing model is the decision tree trained with a minimum node entropy of 0.2. The test accuracy using this is 0.872. 

#### Question

What can you say about the model complexity of the Decision Tree, given the training and validation accuracy? Briefly explain.

[ANSWER GOES HERE]
When the minimum entropy is very low(0.01), the model has very high training accuracy (1.0). However, the validation accuracy is low (0.862) resulting in overfitting. it means that the tree is too complex and is also learning the noise in the dataset. As the minimum entropy starts increasing the training acccuracy starts decreasing slightly and the validation accuracy starts increasing. At the entropy of 2.0, the model underfits the dataset. The entropy of 0.2 gives similar accuracy(0.59) in training and validation set and also the good test accuracy balancing the model complexity and the accuracy. 

## Problem B

Apart from the entropy, another category of Decision Tree algorithm leverages a different metric: Gini Index. The following is the formula of Gini Index: 
       \begin{align}
           Gini = 1 - \sum_i p_i^2,
       \end{align}
       where $p_i$ is the sample as the $p_i$ in entropy that denotes probability of class $i$. You are going to implement *gini\_index()*, and run the same set of experiments again.

In [34]:
# Experiment with different settings of minimum node entropy
candidate_min_entropy = [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1, 2.0]

valid_accuracy = []
for i, min_entropy in enumerate(candidate_min_entropy):
    # Initialize and fit.
    clf = Decision_tree(min_entropy = min_entropy, metric = "gini_index")
    clf.fit(train_x, train_y)
    
    predictions_train = clf.predict(train_x)
    predictions_val = clf.predict(valid_x)
    
    curr_train_accuracy = np.count_nonzero(predictions_train.reshape(-1) == train_y.reshape(-1)) / len(train_x)
    curr_valid_accuracy = np.count_nonzero(predictions_val.reshape(-1) == valid_y.reshape(-1)) / len(valid_x)
    valid_accuracy.append(curr_valid_accuracy)
    
    print(
        f"Training/validation accuracy for minimum gini_index {candidate_min_entropy[i]} "
        f"is {curr_train_accuracy:.3f} / {curr_valid_accuracy}"
    )

# Select the best minimum node entropy and use it to train the model
best_entropy = candidate_min_entropy[np.argmax(valid_accuracy)]
clf = Decision_tree(min_entropy = best_entropy)
clf.fit(train_x, train_y)

# Evaluate on test data
predictions = clf.predict(test_x)
accuracy = np.count_nonzero(predictions.reshape(-1) == test_y.reshape(-1)) / len(test_x)

print(f"Test accuracy with minimum gini_index {best_entropy} is {accuracy:.3f}")

Training/validation accuracy for minimum gini_index 0.01 is 0.999 / 0.8469750889679716
Training/validation accuracy for minimum gini_index 0.05 is 0.990 / 0.8523131672597865
Training/validation accuracy for minimum gini_index 0.1 is 0.978 / 0.8505338078291815
Training/validation accuracy for minimum gini_index 0.2 is 0.948 / 0.8451957295373665
Training/validation accuracy for minimum gini_index 0.5 is 0.800 / 0.7669039145907474
Training/validation accuracy for minimum gini_index 0.8 is 0.384 / 0.39501779359430605
Training/validation accuracy for minimum gini_index 1 is 0.100 / 0.11743772241992882
Training/validation accuracy for minimum gini_index 2.0 is 0.100 / 0.11743772241992882
Test accuracy with minimum gini_index 0.05 is 0.867


#### Question
What can you say about the performance? Are they more or less the same, or which one is better. Briefly explain.

[ANSWER GOES HERE]
What i observed is across all the minimum entropy values, the entropy measure generally has the higher training accuracy as compared to the gini index. In case of the validation accuracy, entropy achives slightly better accuracy. eg at 0.05 minimum entroy, the validation accuracy is 0.862, while at minimum gini index of 0.05, the validation accuracy is 0.852. In case of test accuracy, entropy achieves similar but slightly better test accuracy (0.872 at min_entropy = 0.2) compared to Gini index (0.867 at minimum gini_index of 0.05). As minimum entropy or minimum gini_index increases, both training and validation accuracy drop significantly, indicating underfitting.
Overall, the performance of both entropy and gini index is comparable. Entropy performs slightly better than Gini index in terms of validation and test accuracy, indicating it may create splits that generalize better for this dataset. However, the difference in accuracy is not that drastic. The choice between these would depend on the nature of question/dataset and computational efficiency. 
